In [ ]:
################################
# Intellectual property of Data Science - DMA - BNI. It is intended for use by authorized personnel only (Mentari ER, Atwin P, and Randu H)
# Unauthorized distribution, reproduction, or modification of this code, in whole or in part, is strictly prohibited
# without the express written consent of Data Science - DMA - BNI. Any use of this code outside the scope of its intended
# purpose must be authorized by Data Science - DMA - BNI. This code is provided "as is" without any warranty of any kind,
# express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose,
# and non-infringement. Data Science - DMA - BNI and the authors of this code shall not be liable for any damages, including
# but not limited to direct, indirect, special, incidental, or consequential damages or loss of profit, arising out of
# the use, inability to use, or performance of this code, even if Data Science - DMA - BNI or the authors have been advised
# of the possibility of such damages. By even looking at this code, you agree to these terms and conditions.
##################################

In [1]:
#-------------------------------------#
#         IMPORT DEPENDENCIES         #
#-------------------------------------#
from openai import OpenAI
import os
from tqdm import tqdm
import collections
from datetime import datetime
from locale import setlocale, LC_TIME

import requests
import pandas as pd

import os
import csv
import sys
sys.path.append(os.path.abspath('../../asla'))
from files.glosarium import glosarium

In [3]:
#-----------------------------#
#           FUNCTION          #
#-----------------------------#

def convert_to_indonesian_month(input_date):
    setlocale(LC_TIME, 'id_ID') # Set locale ke bahasa Indonesia
    datetime_object = datetime.strptime(input_date, '%Y-%m') # Mengonversi string input ke objek datetime
    indonesian_date = datetime_object.strftime('%B %Y')

    return indonesian_date
    

filter the original masked file

In [4]:
file_path='../files/masked'
file_name = file_path+"/tren_2.csv"

In [5]:
#EOM dan MTD (avg) next bisa disatuin biarl ebih mudah dipahami DAN hemat karakter

tren_raw=pd.read_csv(f'{file_path}/tren.csv')
glos=glosarium()['tren']
#filter
#filter for dev and prod phase
print(len(tren_raw))
tren=tren_raw[~tren_raw['variable'].isin(['avg interbank','avg intrabank'])] #remove redundant variables
print(len(tren))
tren=tren[tren_raw['periode']!='YEAR TO DATE'] #remove year to date (redundant shortened ver of yoy)
print(len(tren))

#filter for dev phase, minimizing resources
tren=tren[~tren['group_corporate'].str.contains('dummy dont')] #remove records with dummy group_corporate names
print(len(tren))
tren=tren[tren['flag_trx']!='co intra vs co total'] #remove extra variablesco intra vs co total
print(len(tren))
tren=tren[~((tren['flag_trx']=='closedloop')&
        #   (tren['periode']!='YEAR ON YEAR')& #given
          (tren['variable']=='ytd'))] #cuman closed loop, yoy, mtd
print(len(tren))

tren=tren.sort_values(['flag_trx','variable','group_corporate'])

### sampel 10 grup, comment kl ga jadi
sampel_10_grup=list(tren.group_corporate[:10])
tren_sampel_10=tren[tren['group_corporate'].isin(sampel_10_grup)]
tren=tren_sampel_10

tren.to_csv(file_path+"/tren_2.csv",sep=';',index=0)




116074
83727
42119
11201
8942
7783


C:\Users\atwin\AppData\Local\Temp\ipykernel_16568\337184900.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tren=tren[tren_raw['periode']!='YEAR TO DATE'] #remove year to date (redundant shortened ver of yoy)


In [6]:
tren

,Unnamed: 0,row_key,periode,as_of_date,flag_trx,variable,divisi_kelolaan,group_corporate,bulan1,namabulan1,...,namabulan10,bulan11,namabulan11,bulan12,namabulan12,mean_value,min_value,max_value,max_bulan,min_bulan
27728,27728,~YEAR ON YEAR~~bumn & institusi pemerintah~~ B...,YEAR ON YEAR,20230930,CASA,eom,bumn & institusi pemerintah,Bali Bintang Sejahtera,1.966705e+09,2022-10,...,2023-07,1.449231e+10,2023-08,2.467461e+09,2023-09,6.920025e+09,1.966705e+09,1.449231e+10,202308,202210
32576,32576,~YEAR ON YEAR~~retail b~~ Bhakti Agung Propert...,YEAR ON YEAR,20230930,CASA,eom,retail b,Bhakti Agung Propertindo,0.000000e+00,2022-10,...,2023-07,1.158025e+10,2023-08,1.057791e+10,2023-09,7.414475e+09,8.526498e+07,1.158025e+10,202308,202307
32540,32540,~YEAR ON YEAR~~retail b~~ Digital Mediatama Ma...,YEAR ON YEAR,20230930,CASA,eom,retail b,Digital Mediatama Maxima,8.661311e+08,2022-10,...,2023-07,1.379791e+08,2023-08,1.103639e+08,2023-09,3.974347e+08,1.103639e+08,8.911503e+08,202211,202309
27680,27680,~YEAR ON YEAR~~bumn & institusi pemerintah~~ I...,YEAR ON YEAR,20230930,CASA,eom,bumn & institusi pemerintah,Intermedia Capital,1.853568e+10,2022-10,...,2023-07,4.327928e+10,2023-08,4.708300e+10,2023-09,2.681641e+10,1.433320e+10,4.708300e+10,202309,202301
27716,27716,~YEAR ON YEAR~~bumn & institusi pemerintah~~ O...,YEAR ON YEAR,20230930,CASA,eom,bumn & institusi pemerintah,Optima Prima Metal Sinergi,2.087068e+08,2022-10,...,2023-07,1.193156e+10,2023-08,1.225877e+10,2023-09,7.441608e+09,2.087068e+08,1.499610e+10,202211,202210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27726,27726,~YEAR ON YEAR~~bumn & institusi pemerintah~~ O...,YEAR ON YEAR,20230930,closedloop,mtd,bumn & institusi pemerintah,Optima Prima Metal Sinergi,0.000000e+00,2022-10,...,2023-07,7.998000e+00,2023-08,0.000000e+00,2023-09,1.790364e+01,0.000000e+00,1.250440e+02,2022-12,2023-09
27750,27750,~YEAR ON YEAR~~bumn & institusi pemerintah~~ W...,YEAR ON YEAR,20230930,closedloop,mtd,bumn & institusi pemerintah,Widodo Makmur Unggas,4.454800e+01,2022-10,...,2023-07,0.000000e+00,2023-08,0.000000e+00,2023-09,1.784500e+01,0.000000e+00,8.582800e+01,2023-01,2023-09
14781,14781,~YEAR ON YEAR~~corporate a~~ABM Investama~~202...,YEAR ON YEAR,20230930,closedloop,mtd,corporate a,ABM Investama,1.286500e+01,2022-10,...,2023-07,1.560400e+01,2023-08,6.872400e+01,2023-09,5.167442e+01,1.211800e+01,8.300000e+01,2023-07,2022-11
1424,1424,~YEAR ON YEAR~~corporate a~~ADICON JAYA Panca ...,YEAR ON YEAR,20230930,closedloop,mtd,corporate a,ADICON JAYA Panca Budi Idaman,0.000000e+00,2022-10,...,2023-07,1.660000e+02,2023-08,1.660000e+02,2023-09,7.063300e+01,7.470000e-01,1.660000e+02,2023-06,2022-12


In [ ]:
#-------------------------------------------------------#
#           TURN DATA INTO EXPECTED SENTENCES          #
#-------------------------------------------------------#
"Divisi retail a mengelola grup-grup berikut: Suryamas, Trimegah, dlll." 
#  Output
output_file = file_path+'\material_masked_retail.txt'

In [ ]:
#-------------------------------------------------------#
#           TURN DATA INTO EXPECTED SENTENCES          #
#-------------------------------------------------------#
"CASA: rekening bank mencakup current (transaksi) dan savings (tabungan). Meliputi rekening-rekening  Taplus, Taplus Muda, ….. Eom: niali posisi pada tanggal terakhir setiap bulannya Avg intrabank: nilai rata-rata transaksi ke bank luar (selain BNI). " 

In [10]:
#-------------------------------------------------------#
#           TURN DATA INTO EXPECTED SENTENCES          #
#-------------------------------------------------------#
#  Output
output_file = file_path+'\masked_tren_2_sampled_10group.txt'
description='This text describes the financial situations of corporate groups in BNI for each of the past 12 months'

def date_loop(num):
    monthly_dates = []
    for j in range(9, 31, 2): # header kolom 9-31
        date = rows[num][j] # '2022-11'
        converted_date = convert_to_indonesian_month(date) # November 2022
        value = round(float(rows[num][j-1]),1)
        monthly_dates.append(f" {converted_date}: {value}")
    return monthly_dates

with open(file_name, 'r') as file:
    csv_reader = csv.reader(file, delimiter=';')

    # Read by row 
    header = next(csv_reader)
    rows = list(csv_reader)

with open(output_file, 'w') as output:
    last_flag_trx_var=''
    nomor_kategori=1
    for i in range(0, len(rows), 1):  
        converted_dates = []
        variable=rows[i][5]
        flag_trx=rows[i][4]
        grup=rows[i][7]

        flag_trx_var=flag_trx+' '+variable
        if last_flag_trx_var!=flag_trx_var:
            if last_flag_trx_var=='': 
                result_str=description+'\n\n'
            else: 
                result_str=''
                nomor_kategori+=1
            result_str+=flag_trx_var+':\n' 
            last_flag_trx_var=flag_trx_var
        else:
            result_str=''
        result_str+= f"grup {grup} pada"

        for key in glos.keys():
            result_str = result_str.replace(key, glos[key])
        output.write(result_str)
        converted_dates.append(date_loop(i))
    
        for dt in converted_dates:
            details = ', '.join(dt) 
            output.write(details + '\n') 


In [8]:
output

<_io.TextIOWrapper name='../files/masked\\masked_tren_2_sampled_10group.txt' mode='w' encoding='cp1252'>

'Divisi retail a mengelola grup-grup berikut: Suryamas, Trimegah, dlll.'

'CASA: rekening bank mencakup current (transaksi) dan savings (tabungan). Meliputi rekening-rekening  Taplus, Taplus Muda, ….. Eom: niali posisi pada tanggal terakhir setiap bulannya Avg intrabank: nilai rata-rata transaksi ke bank luar (selain BNI). '

In [12]:
#-----------------------------#
#           TESTING           #
#-----------------------------#
#chat gpt baca txt, path
training_file = file_path+'\masked_tren.txt' 
# print(training_file)

with open(training_file,'r') as file:
    


C:\Users\50287\Documents\ASLA_OpenAI\masked\masked_tren.txt
